In [1]:
import pandas as pd
import numpy as np
import geopandas  as gpd
import altair as alt
from datetime import timedelta
from datetime import datetime as dt
import datetime
import json
from altair import datum

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:

#highlighting important NPIs in WA
data = {'date': [ "2020-03-23", "2020-06-01", "2020-11-18", "2021-02-14"], 'event':[ "Stay at home", "Stay at home lifted", "Closing restaurants", "Reopening restaurants"]}

npidf = pd.DataFrame(data)
npidf.date = pd.to_datetime(npidf.date)

rule = alt.Chart(npidf).mark_rule(
    color="black",
    strokeWidth=2, 
    opacity = 0.3
).encode(
    alt.X('date:T', axis=alt.Axis(title=None))
).properties(
    width=800,
    height=300
)

text = alt.Chart(npidf).mark_text(
    align='left',
    baseline='middle',
    dx=2,
    dy=-135,
    size=13
).encode(
    alt.X('date:T',axis=alt.Axis(title=None)),
    text='event',
    color=alt.value('#000000')
).properties(
    width=800,
    height=300
)

In [3]:
#making background shading to represent different variants
cutoff = pd.DataFrame({
     "variant" : ["ancestral", "Alpha", "Delta", "Omicron"],
    'start': ["2020-02-01","2021-03-29", "2021-06-25", "2021-12-02"],
    'stop': ["2021-04-05","2021-06-28", "2021-12-31", "2022-03-8"],
})

areas_bot = alt.Chart(
    cutoff.reset_index()
).mark_rect(
    opacity=0.1
).encode(
    x='start:T',
    x2='stop:T',
    y=alt.value(0),  # pixels from top
    y2=alt.value(-20),  # pixels from top
   
).transform_filter((datum.variant != "Alpha") & (datum.variant != "Omicron"))


areas_top = alt.Chart(
    cutoff.reset_index()
).mark_rect(
    opacity=0.1
).encode(
    x='start:T',
    x2='stop:T',
    y=alt.value(-10),  # pixels from top
    y2=alt.value(-30),  # pixels from top
   
).transform_filter((datum.variant != "ancestral") & (datum.variant != "Delta"))

text_variant_bot = alt.Chart(cutoff.reset_index()).mark_text(
    align='left',
    baseline='middle',
    dx=2,
    dy=-155,
    size=13
).encode(
    alt.X('start:T',axis=alt.Axis(title=None)),
    text='variant',
    color=alt.value('#000000')
).properties(
    width=800,
    height=300
).transform_filter((datum.variant != "Alpha") & (datum.variant != "Omicron"))

text_variant_top = alt.Chart(cutoff.reset_index()).mark_text(
    align='left',
    baseline='middle',
    dx=2,
    dy=-175,
    size=13
).encode(
    alt.X('start:T',axis=alt.Axis(title=None)),
    text='variant',
    color=alt.value('#000000')
).transform_filter((datum.variant != "ancestral") & (datum.variant != "Delta"))



variant_freq = text_variant_bot + text_variant_top + areas_bot + areas_top




In [4]:
ne_df = pd.read_csv("../data-files/ne_df.csv") #exported from ../post_process/plotting_ne_and_mr.ipynb
predictor_df = pd.read_csv("../data-files/predictor_coefficients.csv") #exported from ../post_process/estimating_predictor_coefficients.ipynb

In [5]:
line = alt.Chart(ne_df).mark_area(interpolate='monotone').encode(
    alt.X('date:T', axis=alt.Axis(title="Date", tickCount = "month", grid=False, format="%B %Y")),
    alt.Y('lower_hpd_log_50',axis=alt.Axis(title="Effective Population Size", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_log_50' ),
    color=alt.Color('deme:N')
).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.upper_hpd_log_50 < 95)
)

band = alt.Chart(ne_df).mark_area(
    opacity=0.3, interpolate='monotone'
).encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_log_95'),#axis=None),#, scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_log_95'),
    color=alt.Color('deme:N', legend=None)
).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.upper_hpd_log_95 < 95)
)

band + line

alt.LayerChart(...)

In [6]:
ne = (band + line + text + rule + variant_freq); ne

alt.LayerChart(...)

In [7]:
ne_pred = predictor_df.iloc[1:6]
ne_pred = ne_pred.replace(["Ne_hosp_orig", "Ne_hosp_1wk_fwd", "Ne_hosp_2wk_fwd", "percent_immunity", "cases"], ["Hospializations (no lag)","Hospializations (1wk lag)", "Hospializations (2wk lag)", "Percent Immunity", "Cases" ])

mig_pred = predictor_df.iloc[-2:]
mig_pred = mig_pred.replace(["NPI_dates", "safegraph_between_total_mvmt"], ["NPI Dates", "Safegraph Mobility"])
mig_pred

,Unnamed: 0,deme,mean_Ne_log,mean_Ne_linear,median_Ne_log,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,upper_hpd_linear,lower_hpd_linear
7,0,NPI Dates,0.123847,1.329985,0.0,1.149270,-0.604153,0.0,0.0,14.101638,0.248798
8,0,Safegraph Mobility,0.016028,1.037596,0.0,0.286016,-0.121449,0.0,0.0,1.932041,0.756051


In [8]:
# error_bars = alt.Chart(mig_short).mark_errorbar(extent='ci').encode(
#   x=alt.X('mig_per_bl:Q', scale=alt.Scale(zero=False)),
#   y=alt.Y('migration_direction:N')
# )

points = alt.Chart(ne_pred).mark_point(filled=True, color='black', width = 5).encode(
  y=alt.Y('mean_Ne_log:Q',axis = alt.Axis(title = "Coefficient", grid =False,) ),
  x=alt.X('deme', title = "Predictor",   axis=alt.Axis(labelLimit = 0, labelAngle=-35)),
).transform_filter(datum.predictor != "Clock" or datum.predictor != "air").properties(
    width=650,
    height=200
)

hpd = alt.Chart(ne_pred).mark_rule(strokeWidth = 2).encode(
    y2=alt.Y2("upper_hpd_log_95"),
    y = alt.Y("lower_hpd_log_95"),
    x=alt.X("deme")
).transform_filter(datum.predictor != "Clock").properties(
    width=850,
    height=300
)

one_line = alt.Chart(pd.DataFrame({'y': [0.0]})).mark_rule(strokeDash=[1,1]).encode(y='y').properties(
    width=850,
    height=300
)

predict_coeff_ne = (points + hpd + one_line ).properties(title='B')
# predict_coeff_ne.configure_view(
#     strokeWidth=0
# ).configure_title(
#     anchor='start', fontSize= 35
# ).configure_axis(
#     labelFontSize=60,
#     titleFontSize=60
# ).configure_legend(
#     labelFontSize = 60)

predict_coeff_ne

alt.LayerChart(...)

In [9]:
# error_bars = alt.Chart(mig_short).mark_errorbar(extent='ci').encode(
#   x=alt.X('mig_per_bl:Q', scale=alt.Scale(zero=False)),
#   y=alt.Y('migration_direction:N')
# )

points = alt.Chart(mig_pred).mark_point(filled=True, color='black').encode(
  y=alt.Y('mean_Ne_log:Q',axis = alt.Axis(title = "Coefficient", grid =False, labelFontSize= 18,titleFontSize= 16) ),
  x=alt.X('deme', title = "Predictor",  axis=alt.Axis(labelFontSize= 16,titleFontSize= 16, labelAngle=-35)),
).transform_filter(datum.predictor != "Clock" ).properties(
    width=250,
    height=200
)

hpd = alt.Chart(mig_pred).mark_rule().encode(
    y2=alt.Y2("upper_hpd_log_95"),
    y = alt.Y("lower_hpd_log_95"),
    x=alt.X("deme")
).transform_filter(datum.predictor != "Clock").properties(
    width=300,
    height=150
)

one_line = alt.Chart(pd.DataFrame({'y': [0.0]})).mark_rule(strokeDash=[1,1]).encode(y='y').properties(
    width=300,
    height=150
)

predict_coeff_mig = (points + hpd + one_line ).properties(title='C')
predict_coeff_mig.configure_view(
    strokeWidth=0
).configure_title(
    anchor='start', fontSize= 25
).configure_axis(
    labelFontSize=24,
    titleFontSize=24
).configure_legend(
    labelFontSize = 60)




alt.LayerChart(...)

In [10]:
# Add labels to each chart
ne = ne.properties(title='A')


# Concatenate labeled charts B and C horizontally
predictors  = alt.hconcat(predict_coeff_ne , predict_coeff_mig).resolve_scale(color = "independent")

# Concatenate labeled charts AB and CD vertically
chart_abcd_labeled = alt.vconcat(ne, predictors, spacing=50)#.resolve_scale(x='shared', color ='shared')

chart_abcd_labeled.configure_view(
    strokeWidth=0
).configure_title(
    anchor='start', fontSize= 30
).configure_axis(
    labelFontSize=14,
    titleFontSize=14
).configure_legend(
    labelFontSize = 14)


alt.VConcatChart(...)